In [1]:
import sys
print(sys.version)

3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:40:08) [MSC v.1938 64 bit (AMD64)]


In [ ]:
# Require python 3.10
#pip install torch torchvision transformers sentencepiece accelerate
#
#llama model list
##llama download --source meta --model-id CHOSEN_MODEL_ID
#llama download --source meta --model-id Llama3.2-3B-Instruct:int4-qlora-eo8
#pip install torch fairscale fire blobfile
## verifying download 
#llama model verify-download --model-id Llama3.2-3B-Instruct:int4-qlora-eo8
#
## Setting env variable 
#$env:CHECKPOINT_DIR = "C:\Users\Zunaira\.llama\checkpoints\Meta-Llama3.2-3B-Instruct"



In [ ]:
#pip install llama-stack


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


In [6]:
model_id = "C:\\Users\\Zunaira\\.llama\\checkpoints\\Llama3.2-3B-Instruct-int4-qlora-eo8"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.float16,  # Use float16 for efficiency
    device_map="auto"          # Automatically assign model to available devices (CPU/GPU)
)


OSError: C:\Users\Zunaira\.llama\checkpoints\Llama3.2-3B-Instruct-int4-qlora-eo8 does not appear to have a file named config.json. Checkout 'https://huggingface.co/C:\Users\Zunaira\.llama\checkpoints\Llama3.2-3B-Instruct-int4-qlora-eo8/tree/None' for available files.

In [ ]:
prompt = "Explain the concept of machine learning."
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(inputs.input_ids, max_length=100, num_return_sequences=1)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


In [ ]:
#!/bin/bash

#CHECKPOINT_DIR=~/.llama/checkpoints/Meta-Llama3.1-8B-Instruct
#PYTHONPATH=$(git rev-parse --show-toplevel) torchrun llama_models/scripts/example_chat_completion.py $CHECKPOINT_DIR

SyntaxError: invalid decimal literal (2685098049.py, line 3)